In [ ]:
import pandas as pd
from ast import literal_eval
books_df = pd.read_csv('data/books_enriched.csv',  index_col=[0], converters={"genres": literal_eval})
ratings = pd.read_csv('data/ratings.csv', converters={"genres": literal_eval})

In [ ]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5976479 entries, 0 to 5976478
Data columns (total 3 columns):
 #   Column   Dtype
---  ------   -----
 0   user_id  int64
 1   book_id  int64
 2   rating   int64
dtypes: int64(3)
memory usage: 136.8 MB


In [ ]:
#book_cols= ['book_id', 'title', 'authors','original_publication_year','genres','pages','average_rating','description']
book_cols= ['book_id', 'title', 'authors','genres','average_rating']
books_df = books_df[book_cols]

In [ ]:
book_rating = ratings.merge(books_df, on='book_id')

# Baseline model

In [ ]:
book_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5976479 entries, 0 to 5976478
Data columns (total 7 columns):
 #   Column          Dtype  
---  ------          -----  
 0   user_id         int64  
 1   book_id         int64  
 2   rating          int64  
 3   title           object 
 4   authors         object 
 5   genres          object 
 6   average_rating  float64
dtypes: float64(1), int64(3), object(3)
memory usage: 319.2+ MB


In [ ]:
class Dataset():
  def __init__(self, dataset):
        self.df = dataset
        train_data, test_data = self.train_test_split()

  def train_test_split(self, test_size=0.2, random_state=42, stratify=None):
    from sklearn.model_selection import train_test_split
    train_data, test_data = train_test_split(self.df, test_size=test_size, random_state=random_state, stratify=stratify)
    return train_data, test_data
  def get_user_ids(self, data):
    return data['user_id'].unique()
  def get_book_ids(self, data):
    return data['book_id'].unique()

In [ ]:
import numpy as np
import math

class RandomRecommender:
    def __init__(self, train_data):
        self.train_data = train_data
        self.book_ids = np.unique(train_data['book_id'])

    def predict(self, user_id, k=10):
        return np.random.choice(self.book_ids, size=k, replace=False)

    def evaluate(self, test_data, k=10):
        user_ids = np.unique(test_data['user_id'])
        hit_rate = []
        ndcg_scores = []
        count=0
        for user_id in user_ids:
            true_books = test_data[test_data['user_id'] == user_id]['book_id'].unique()
            pred_books = self.predict(user_id, k)

            hits = len(set(true_books) & set(pred_books))
            hit_rate.append(hits / min(k, len(true_books)))

            # Calculate DCG
            dcg = sum([1 / math.log2(rank + 2) if pred_books[rank] in true_books else 0 for rank in range(k)])
            # Calculate IDCG
            idcg = sum([1 / math.log2(i + 2) for i in range(min(k, len(true_books)))])
            # Calculate NDCG
            ndcg = dcg / idcg if idcg > 0 else 0
            ndcg_scores.append(ndcg)
            count+=1
            if count==5000:
              break

        avg_hit_rate = np.mean(hit_rate)
        avg_ndcg = np.mean(ndcg_scores)
        return avg_hit_rate, avg_ndcg

In [ ]:
train,test= Dataset(book_rating).train_test_split()

In [ ]:
recommender = RandomRecommender(train)
hit_rate, ndcg = recommender.evaluate(test, k=10)
print(f"Hit Rate@10: {hit_rate:.4f}")
print(f"NDCG : {ndcg:.4f}")

Hit Rate@10: 0.0023
NDCG : 0.0024


In [ ]:
import numpy as np
import math

class PopularityRecommender:
    def __init__(self, train_data):
        self.train_data = train_data
        self.popular_books = self.get_popular_books()

    def get_popular_books(self, n=1000):
        book_counts = self.train_data['book_id'].value_counts()
        popular_books = book_counts.index[:n].tolist()
        return popular_books

    def predict(self, user_id, k=10):
        return np.random.choice(self.popular_books, size=k, replace=False)

    def evaluate(self, test_data, k=10):
        user_ids = np.unique(test_data['user_id'])
        hit_rate = []
        ndcg_scores = []
        print(len(user_ids))
        count=0
        for user_id in user_ids:
            true_books = test_data[test_data['user_id'] == user_id]['book_id'].unique()
            pred_books = self.predict(user_id, k)

            hits = len(set(true_books) & set(pred_books))
            hit_rate.append(hits / min(k, len(true_books)))

            # Calculate DCG
            dcg = sum([1 / math.log2(rank + 2) if pred_books[rank] in true_books else 0 for rank in range(k)])
            # Calculate IDCG
            idcg = sum([1 / math.log2(i + 2) for i in range(min(k, len(true_books)))])
            # Calculate NDCG
            ndcg = dcg / idcg if idcg > 0 else 0
            ndcg_scores.append(ndcg)
            count+=1
            if count==5000:
              break
        avg_hit_rate = np.mean(hit_rate)
        avg_ndcg = np.mean(ndcg_scores)
        return avg_hit_rate, avg_ndcg

In [ ]:
import time

start_time = time.time()

recommender = PopularityRecommender(train)

hit_rate, ndcg = recommender.evaluate(test, k=10)
print(f"Hit Rate@50: {hit_rate:.4f}")
print(f"NDCG : {ndcg:.4f}")

end_time = time.time()
elapsed_time = end_time - start_time

print(f"Elapsed time: {elapsed_time:.4f} seconds")

53424
Hit Rate@50: 0.0137
NDCG : 0.0134
Elapsed time: 14.6640 seconds


# Content Based Filtering with Description Similarity

In [ ]:
import pandas as pd
from ast import literal_eval
books_df = pd.read_csv('data/books_enriched.csv',  index_col=[0], converters={"genres": literal_eval})
ratings = pd.read_csv('data/ratings.csv', converters={"genres": literal_eval})

In [ ]:

book_cols= ['book_id', 'title','description']
books_df = books_df.dropna(subset=['description'])
books_df = books_df[book_cols]
book_rating = ratings.merge(books_df, on='book_id')


In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

book_cols = ['book_id', 'title', 'description']
books_df = books_df.dropna(subset=['description'])
books_df = books_df[book_cols]
book_rating = ratings.merge(books_df, on='book_id')

books_df['description'] = books_df['description'].str.lower().str.replace('[^a-zA-Z0-9\s]', '', regex=True)

books_df = books_df.reset_index(drop=True)

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(books_df['description'])

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

similar_books_data = []

for idx, row in books_df.iterrows():
    book_id = row['book_id']
    similar_indices = cosine_sim[idx].argsort()[::-1][1:5]
    similar_book_ids = books_df.iloc[similar_indices]['book_id'].tolist()
    similar_books_data.append({'book_id': book_id, 'similar_books': similar_book_ids})

similar_books_df = pd.DataFrame(similar_books_data)

book_rating = book_rating.merge(similar_books_df, on='book_id', how='left')



In [ ]:
book_rating.similar_books[0]

[2833, 3015, 4393, 1467]

In [ ]:
class ContentBasedRecommender:
    def __init__(self, train_data):
        self.train_data = train_data

    def predict(self, user_id, k=10):
        user_books = set(self.train_data[self.train_data['user_id'] == user_id]['book_id'])
        similar_books = set().union(*(self.train_data[self.train_data['book_id'] == book_id]['similar_books'].iloc[0] for book_id in user_books))
        recommended_books = list(similar_books - user_books)

        return np.random.choice(recommended_books, size=k, replace=False) if len(recommended_books) >= k else recommended_books

    def evaluate(self, test_data, k=10):
        user_ids = test_data['user_id'].unique()
        hit_rate, ndcg_scores = [], []

        for user_id in user_ids[:100]:
            true_books = set(test_data[test_data['user_id'] == user_id]['book_id'])
            pred_books = set(self.predict(user_id, k))

            hits = len(true_books & pred_books)
            hit_rate.append(hits / min(k, len(true_books)))

            dcg = sum(1 / math.log2(rank + 2) for rank, book in enumerate(pred_books) if book in true_books)
            idcg = sum(1 / math.log2(i + 2) for i in range(min(k, len(true_books))))
            ndcg = dcg / idcg if idcg > 0 else 0
            ndcg_scores.append(ndcg)

        return np.mean(hit_rate), np.mean(ndcg_scores)




In [ ]:
train,test= Dataset(book_rating).train_test_split()

In [ ]:
import time

start_time = time.time()

recommender = ContentBasedRecommender(train)

hit_rate, ndcg = recommender.evaluate(test, k=10)
print(f"Hit Rate@50: {hit_rate:.4f}")
print(f"NDCG : {ndcg:.4f}")

end_time = time.time()
elapsed_time = end_time - start_time

print(f"Elapsed time: {elapsed_time:.4f} seconds")

Hit Rate@50: 0.0350
NDCG : 0.0327
Elapsed time: 106.2988 seconds


In [ ]:
import pickle


with open("content_based_recommender.pkl", "wb") as f:
    pickle.dump(recommender, f)

In [ ]:
with open('content_based_recommender.pkl', 'rb') as file:
    loaded_recommender = pickle.load(file)

In [ ]:
loaded_recommender.predict(18)

array([8797, 5966, 4611, 6066, 2539,  247, 4575, 5860, 5486, 5961])